# TUI Template - Design Only

Prototype for the TUI design. Exported to `tui_writer/tui_template.py` but NOT part of main package.

*Always nbdev_export after any changes*

**Run with:** `python -m tui_writer.tui_template`


In [ ]:
#| default_exp tui_template

## CSS Styles & Layout System

The following cell contains all CSS styling for the TUI application. This defines:

- **Grid Layout**: 4×2 grid system for the top cards section
- **Responsive Design**: Components adapt to terminal size  
- **State-based Styling**: CSS classes (`.active`, `.paused`) control button visibility and colors
- **Component Borders**: Each widget has distinct colored borders for visual separation
- **Button Controls**: Media buttons show/hide based on recording state

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        background: $surface;
    }
    Header {
        background: $primary;
    }

    StatusCard {
        width: auto;
        row-span: 2;
        border: solid $primary;
        border-title-align: center;
        height: 100%;
        padding: 1 0;
    }

    InfoCard {
        width: auto;
        border: solid #6b7280;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }

    TranscriptField {
        width: auto;
        border: solid $warning;
        border-title-align: center;
        height: 100%;
        column-span: 3;
        padding: 1;
    }
    TranscriptField TextArea {
        border: none;
    }

    #top-cards {
        layout: grid;
        grid-size: 4 2;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
        margin: 1;
        height: 40vh;
    }

    TextField {
        width: 100%;
        margin: 1;
        padding: 1;
        border: solid #8b5cf6;
        border-title-align: center;
        height: 50vh;
    }

    #status {
        content-align: center middle;
    }

    #status-time {
        border: double green;
        width: auto;
        padding: 0 1;
        margin: 1 0;
    }

    #media-controls {
        dock: bottom;
        align: center middle;
    }

    #media-controls Button {
        min-width: 20;
    }

    #pause,
    #resume,
    #stop {
        display: none;
    }

    .active #start {
        display: none;
    }
    .active #pause,
    .active #stop {
        display: block;
    }
    .active #status {
        color: $error;
    }

    .paused #start {
        display: none;
    }
    
    .paused #status {
        color: $warning;
    }

    .paused #resume,
    .paused #stop {
        display: block;
    }
"""

## Core Imports & Dependencies

This cell imports all the necessary Textual framework components and Python standard library modules:

- **Textual Core**: `App`, `ComposeResult` for the main application structure
- **Reactive System**: `reactive` decorator for automatic UI updates when data changes
- **Layout Containers**: `Container`, `Grid`, `HorizontalGroup` for organizing widgets
- **UI Widgets**: `Header`, `Footer`, `Button`, `TextArea`, `Static` for interface elements
- **Event System**: `@on` decorator for handling user interactions
- **Timing**: `monotonic`, `Enum` for precise time tracking and state management

In [ ]:
#| export
from time import monotonic
from enum import Enum
from textual import on
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, VerticalGroup, Grid, HorizontalGroup, CenterMiddle
from textual.widgets import Header, Footer, Static, Digits, Button, Label, TextArea

## Widget Classes - UI Components

The following cell defines all the custom widget classes that make up the TUI interface:

### TimeDisplay Widget
- **Purpose**: High-precision timer showing elapsed recording time (updates 60x per second)
- **Key Methods**: `start()`, `pause()`, `stop()` - control the timer state
- **Reactive Property**: `time_elapsed` automatically updates the display
- **Features**: Accumulates time across pause/resume cycles for accurate tracking

### StatusCard Widget  
- **Purpose**: Main control panel showing recording status and media control buttons
- **Key Methods**: `start_transcript()`, `pause_transcript()`, `resume_transcript()`, `stop_transcript()`
- **CSS Classes**: Dynamically adds `.active`/`.paused` classes to control button visibility
- **Layout**: Status text, timer display, and 4 control buttons (Record, Pause, Resume, Stop)

### InfoCard Widget
- **Purpose**: Displays system information (AI model, microphone settings)
- **Simple Implementation**: Static information display for configuration details

### TranscriptField Widget  
- **Purpose**: Shows the most recent transcript block from live speech recognition
- **Reactive Property**: `transcript_block` will update when new transcription arrive
- **Read-only**: Users can't edit this - it's updated automatically by the transcription system

### TextField Widget
- **Purpose**: Main text editing area where all transcribed content accumulates
- **Key Method**: `watch_content()` syncs the TextArea when reactive content changes
- **Reactive Property**: `content` - the main text that gets built up over time
- **Auto-scroll**: Automatically scrolls to bottom when new content is added

In [ ]:
#| export

class TimeDisplay(Digits):
    """Custom time display widget"""
    
    accumulated_time = 0
    time_elapsed = reactive(0)

    def on_mount(self):
        self.update_timer = self.set_interval(
            1 / 60, 
            self.update_time_elapsed,
            pause=True    
        )


    def update_time_elapsed(self):
        self.time_elapsed = self.accumulated_time + (monotonic() - self.start_time)

    def watch_time_elapsed(self):
        time = self.time_elapsed
        time, seconds = divmod(time, 60)
        hours, minutes = divmod(time, 60)
        time_string = f"{hours:02.0f}:{minutes:02.0f}:{seconds:05.2f}"
        self.update(time_string)

    def start(self):
        """Start keeping track of time elapsed"""
        self.start_time = monotonic()
        self.update_timer.resume()
    
    def pause(self):
        """Pause keeping track of time elapsed"""
        self.accumulated_time = self.time_elapsed
        self.update_timer.pause()

    def stop(self):
        """Stop/reset the time elapsed"""
        self.start_time = monotonic()
        self.accumulated_time = 0
        self.time_elapsed = 0
        self.update_timer.pause()

class StatusCard(Label):
    """Recording status card with controls"""
    status_text = reactive("● STANDBY")

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Recording Status"

    def start_transcript(self):
        self.add_class("active")
        self.status_text = "● RECORDING..."

    def pause_transcript(self):
        self.remove_class("active")
        self.add_class("paused")
        self.status_text = "⏸ PAUSED"

    def resume_transcript(self):
        self.remove_class("paused")
        self.add_class("active")
        self.status_text = "● RECORDING..."

    def stop_transcript(self):
        self.remove_class("paused")
        self.remove_class("active")
        self.status_text = "● STANDBY"

    def watch_status_text(self):
        """Update the status display when status_text changes"""
        if self.is_mounted:
            self.query_one("#status").update(self.status_text)

    def compose(self) -> ComposeResult:
        yield Static(self.status_text, id="status")
        with CenterMiddle():
            yield TimeDisplay("0:00:00", id="status-time")
        with HorizontalGroup(id="media-controls"):
            yield Button.error("● REC <space>", id="start")
            yield Button.success("⏸ PAUSE <space>", id="pause")
            yield Button.warning("▶ RESUME <space>", id="resume")
            yield Button("◼ STOP", id="stop")
    
class InfoCard(Label):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Info"

    def compose(self) -> ComposeResult:
        yield Static("Model: Whisper-Tiny")
        yield Static("Microphone: Default System Mic")

class TranscriptField(Label):

    transcript_block = reactive("", always_update=True)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Last Transcript-block"

    def compose(self) -> ComposeResult:
        yield TextArea("Change the name to Jens William", read_only=True, show_cursor=False)


class TextField(Label):

    content = reactive("Default text\n")

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.border_title = "Text Field"

    def compose(self) -> ComposeResult:
        yield TextArea(
            "",
            read_only=True,
            show_line_numbers=True,
            show_cursor=False,
            id="main-textarea"
        )

    def watch_content(self) -> None:
        """This method is called whenever 'content' changes"""
        if self.is_mounted:
            textarea = self.query_one("#main-textarea")
            textarea.text = self.content
            textarea.refresh()  # This forces the TextArea to redraw!
            textarea.scroll_end()

## Main Application Class - App Control & Flow

This cell contains the `TUITemplate` class - the heart of the application that coordinates everything:

### Recording State Management
- **RecordingState Enum**: `STANDBY`, `RECORDING`, `PAUSED` - tracks the current app state
- **Central State Control**: All state changes go through the main app class

### App Structure & Layout
- **`compose()` Method**: Defines the UI layout - header, 4×2 grid, text field, footer
- **Key Bindings**: Space bar, 'q' for quit, 'l' for list models
- **Widget Coordination**: App controls all child widgets through direct method calls

### **🎯 MAIN START/STOP FLOW - Where the App Really Starts**

The app's core functionality centers around these main methods that coordinate everything:

#### `start_recording()` - THE MAIN START METHOD
- **What it does**: This is where recording actually begins
- **Coordinates**: Updates app state → starts StatusCard → starts TimeDisplay timer
- **Triggered by**: Both the "REC" button click AND the space bar keyboard shortcut

#### `pause_recording()` & `resume_recording()` - PAUSE/RESUME FLOW  
- **What they do**: Pause/resume the recording session while maintaining state
- **Coordinates**: Updates app state → pauses StatusCard → pauses TimeDisplay
- **Triggered by**: Both button clicks AND space bar (depending on current state)

#### `stop_recording()` - THE MAIN STOP METHOD
- **What it does**: This is where recording ends and content is finalized
- **Coordinates**: Resets app state → stops StatusCard → stops TimeDisplay → processes final content
- **Triggered by**: Both the "STOP" button click AND when user wants to end session

### **🔗 Keyboard vs Button Control - Same Methods!**

**Important**: The keyboard shortcuts (`action_toggle_status()`) and button clicks (`@on(Button.Pressed)`) **call the exact same methods**:

- **Space Bar** → `action_toggle_status()` → `start_recording()` / `pause_recording()` / `resume_recording()`
- **"REC" Button** → `@on(Button.Pressed, "#start")` → `start_recording()`  
- **"PAUSE" Button** → `@on(Button.Pressed, "#pause")` → `pause_recording()`

This ensures consistent behavior whether users click buttons or use keyboard shortcuts!

In [ ]:
#| export

class RecordingState(Enum):
    STANDBY = "standby"
    RECORDING = "recording"
    PAUSED = "paused"

class TUITemplate(App):
    """Standalone TUI template for design testing."""

    TITLE = "TUI Writer"
    SUB_TITLE = "Design Template"
    CSS = TEXTUAL_CSS

    BINDINGS = [
        ("q", "quit", "Quit"),
        ("space", "toggle_status", "Start/Pause/Resume Recording"),
        ("l", "list_models", "List Models")
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.recording_state = RecordingState.STANDBY

    def compose(self) -> ComposeResult:
        header = Header(show_clock=True)
        header.tall = True
        yield header

        with Container(id="top-cards"):
            yield StatusCard()
            yield InfoCard()  # Add an Info widget here
            yield TranscriptField()
        yield TextField()
        yield Footer()

    # Action methods triggered by key bindings, refers to button-methods
    def action_toggle_status(self) -> None:
        """An action to toggle recording status."""
        if self.recording_state == RecordingState.RECORDING:
            self.pause_recording()
        elif self.recording_state == RecordingState.PAUSED:
            self.resume_recording()
        else: # RecordingState.STANDBY
            self.start_recording()
    
    @on(Button.Pressed, "#start")
    def start_recording(self):
        """Centralized method to start recording"""
        self.recording_state = RecordingState.RECORDING

        status_card = self.query_one(StatusCard)
        time_display = self.query_one(TimeDisplay)

        status_card.start_transcript()
        time_display.start()

    @on(Button.Pressed, "#pause")
    def pause_recording(self):
        """Central method to pause recording"""
        self.recording_state = RecordingState.PAUSED

        status_card = self.query_one(StatusCard)
        time_display = self.query_one(TimeDisplay)
        
        status_card.pause_transcript()
        time_display.pause()
    
    @on(Button.Pressed, "#resume")
    def resume_recording(self):
        """Central method to resume recording"""
        self.recording_state = RecordingState.RECORDING

        status_card = self.query_one(StatusCard)
        time_display = self.query_one(TimeDisplay)

        status_card.resume_transcript()
        time_display.start()

    @on(Button.Pressed, "#stop")
    def stop_recording(self):
        """Central method to stop recording"""
        self.recording_state = RecordingState.STANDBY

        status_card = self.query_one(StatusCard)
        time_display = self.query_one(TimeDisplay)

        status_card.stop_transcript()
        time_display.stop()

## Testing the TUI Application Logic

### **🧪 Testing Strategy Overview**

The following test cells demonstrate how to test different aspects of the TUI application without actually running the UI. This is important because the main app methods contain UI-specific calls that require mounted widgets.

### **⚠️ Why App Methods Can't Be Tested Directly**

The main app methods like `start_recording()`, `pause_recording()`, etc. contain calls to `self.query_one()` which require:
- The app to be running (`app.run()`)  
- Widgets to be mounted and available in the DOM
- A full Textual event loop

```python
# This fails in notebook tests:
def start_recording(self):
    self.recording_state = RecordingState.RECORDING
    
    # These lines need mounted widgets - they fail outside app.run()
    status_card = self.query_one(StatusCard)  # ❌ ScreenStackError
    time_display = self.query_one(TimeDisplay)  # ❌ ScreenStackError
```

### **✅ What We Can Test**

The upcoming test cells show different testing strategies:

1. **State Management**: Test the `RecordingState` enum and state transitions
2. **App Configuration**: Test app setup, bindings, and widget creation  
3. **Widget Logic**: Test individual widget behavior (like `TextField.content`)
4. **Integration Patterns**: Test how components would work together

### **🎯 Test Cell Overview**

Each test cell focuses on a specific aspect that can be safely tested in a notebook environment without requiring the full TUI to be running.

In [ ]:
# Test 1: Recording State Functionality Test

# Create an app instance (without running it)
app = TUITemplate()

print(f"Initial recording state: {app.recording_state}\n")

print("Testing state transitions:")

# Test 1: Check initial state
assert app.recording_state == RecordingState.STANDBY
print("✅ Initial state is STANDBY")

# Test 2: We can change the state manually to test the enum
app.recording_state = RecordingState.RECORDING
assert app.recording_state == RecordingState.RECORDING
print("✅ State change to RECORDING works")

# Test 3: Test pause state
app.recording_state = RecordingState.PAUSED
assert app.recording_state == RecordingState.PAUSED
print("✅ State change to PAUSED works")

# Test 4: Test back to standby
app.recording_state = RecordingState.STANDBY
assert app.recording_state == RecordingState.STANDBY
print("✅ State change back to STANDBY works")

## Individual Widget Functionality Tests

These individual cells test each custom Widget's functionality. These methods would be run by the App class's main methods when running the tui_template module in a terminal.

### **Widget Functionality test order:**

1. **```StatusCard()```** 
   - *Tests the ```status_text``` reactive variable, and ```start_transcript()```, ```pause_transcript()```, ```resume_transcript()``` & ```stop_transcript()```*
   - *The x_transcript() methods change the ```status_text``` and add or remove textual css-classes to change the UI*

2. **```TextField()```**
   - *Tests the ```content``` reactive variable, appends text with += and checks that it's not equal to the initial value of ```content```*

3. **```TimeDisplay()```**
   - *Tests the ```time_elapsed``` reactive property and timer formatting logic*
   - *Tests ```start()```, ```pause()```, ```stop()``` methods that control the recording timer*
   - *Uses a MockTimer to simulate the ```update_timer``` that would normally be created in ```on_mount()```*
   - *Validates time formatting: converts seconds to ```HH:MM:SS.SS``` format*
   - *Tests accumulated time tracking across pause/resume cycles*

In [ ]:
# Test StatusCard reactive behavior
status_card = StatusCard()
print(f"Initial status_text: '{status_card.status_text}'")

status_card.start_transcript()
assert "RECORDING" in status_card.status_text
print(f"✅ start_transcript(): '{status_card.status_text}'")

status_card.pause_transcript() 
assert "PAUSED" in status_card.status_text
print(f"✅ pause_transcript(): '{status_card.status_text}'")

status_card.resume_transcript()
assert "RECORDING" in status_card.status_text
print(f"✅ resume_transcript(): '{status_card.status_text}'")

status_card.stop_transcript()
assert "STANDBY" in status_card.status_text
print(f"✅ stop_transcript(): '{status_card.status_text}'")

*When trying to access the TextField()'s reactive ```content``` attribute:*
- **In the actual app:**
    ```python
    self.query_one(TextField).content
    ```

In [ ]:
# Test TextField reactive content
text_field = TextField()
original_content = text_field.content
text_field.content += "New line added!\n"
assert text_field.content != original_content
print(f"✅ TextField content update works")

✅ TextField content update works


In [ ]:
# Test TimeDisplay methods and functionality
print("🧪 Testing TimeDisplay Widget:")
print()

time_display = TimeDisplay()

# Test initial state
assert time_display.time_elapsed == 0
assert time_display.accumulated_time == 0
print(f"✅ Initial time_elapsed: {time_display.time_elapsed}")
print(f"✅ Initial accumulated_time: {time_display.accumulated_time}")

# Test that we can manually set time_elapsed (reactive property)
time_display.time_elapsed = 65.5  # 1 minute, 5.5 seconds
assert time_display.time_elapsed == 65.5
print(f"✅ Can set time_elapsed: {time_display.time_elapsed} seconds")

# Test the watch_time_elapsed formatting (simulating what happens in UI)
print("Testing time formatting logic:")

# Simulate different time values and check formatting
test_times = [0, 30.5, 65.5, 3661.25]  # 0s, 30.5s, 1m5.5s, 1h1m1.25s

for test_time in test_times:
    time_display.time_elapsed = test_time
    
    # Manually run the formatting logic from watch_time_elapsed
    time = time_display.time_elapsed
    time, seconds = divmod(time, 60)
    hours, minutes = divmod(time, 60)
    time_string = f"{hours:02.0f}:{minutes:02.0f}:{seconds:05.2f}"
    
    print(f"  {test_time}s → {time_string}")

print()

# create a mock timer, since TimeDisplay's on_mount method only runs when actually running the app
class MockTimer:
    def __init__(self):
        self.is_paused = True
    
    def pause(self):
        self.is_paused = True
        print("    MockTimer paused")
    
    def resume(self):
        self.is_paused = False
        print("    MockTimer resumed")

# Add the mock timer to our time_display
time_display.update_timer = MockTimer()

# Now we can test the methods!
print("\n1. Testing start() method:")
time_display.start()
assert hasattr(time_display, 'start_time')
assert time_display.update_timer.is_paused == False
print("✅ start() method works - sets start_time and resumes timer")

print("\n2. Testing pause() method:")
time_display.time_elapsed = 5.0  # Simulate 5 seconds elapsed
time_display.pause()
assert time_display.accumulated_time == 5.0
assert time_display.update_timer.is_paused == True
print("✅ pause() method works - accumulates time and pauses timer")

print("\n3. Testing stop() method:")
time_display.stop()
assert time_display.time_elapsed == 0
assert time_display.accumulated_time == 0
assert time_display.update_timer.is_paused == True
print("✅ stop() method works - resets everything and pauses timer")

print("\n🎯 All TimeDisplay methods tested successfully!")

### **🔄 How the Reactive System Works**

When you change `TextField.content`, the `watch_content()` method automatically:
1. Updates the TextArea widget (`#main-textarea`) 
2. Refreshes the display
3. Scrolls to the bottom

### **💡 Why Use the Reactive Property?**

- **Automatic UI Updates**: The `watch_content()` method handles all the TextArea synchronization
- **Consistent State**: The `content` property is the "source of truth" 
- **Future-Proof**: Easy to add features like auto-save, word count, etc.
- **Clean Code**: One line to update, automatic scrolling and refresh

## Application Entry Point

This cell provides the standard Python entry point to run the TUI application:

- **`if __name__ == "__main__"`**: Ensures the app only runs when the file is executed directly
- **`app = TUITemplate()`**: Creates an instance of our main application class  
- **`app.run()`**: Starts the Textual event loop and displays the TUI

**Run this with:** `python -m tui_writer.tui_template`

In [ ]:
#| export
if __name__ == "__main__":
    app = TUITemplate()
    app.run()